In [1]:
import numpy as np
import torch
from torch.cuda.amp import autocast
from torch.utils.data import DataLoader
from tqdm import tqdm
from utils.preprocessing import crop_center
from models.sfcn_original import SFCN
from utils.datasets import TorchDataset as TD
import torch.nn.functional as F


In [2]:
mode = 'test'
dataloader = DataLoader(TD(f'/data/Data/PD/{mode}'), batch_size=8, shuffle=False)

In [3]:
next(iter(dataloader))[0][0].shape

torch.Size([1, 160, 192, 160])

In [4]:
model = SFCN(output_dim=2, channel_number=[28, 58, 128, 256, 256, 64])
model = torch.nn.DataParallel(model)
fp_ = 'checkpoints/sfcn-pretrained/run_20191008_00_epoch_last.p'
model.load_state_dict(torch.load(fp_))
model.cuda()
model.eval() # Don't forget this. BatchNorm will be affected if not in eval mode.

y_true = []
y_pred = []
# Example data: some random brain in the MNI152 1mm std space
for batch in tqdm(dataloader):
    y_true += batch[2].cpu().tolist()
    input_data =batch[0]

    with torch.no_grad():
        output = model(input_data)

    y_pred += output

100%|██████████| 33/33 [00:05<00:00,  5.78it/s]


In [5]:
y_pred[0].shape

torch.Size([2, 1, 1, 1])

In [6]:
y_pred = np.exp(np.vstack(y_pred))
y_pred_bin = np.argmax(y_pred, axis=1)
y_true = np.array(y_true)
np.mean(y_pred_bin==y_true)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [28]:
np.max(y_pred_bin)

0